<a href="https://colab.research.google.com/github/melinadwisafitri/mental-care/blob/machine-learning/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request
import json

import nltk
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
data_url = 'https://raw.githubusercontent.com/melinadwisafitri/mental-care/machine-learning/dataset/data.json'
urllib.request.urlretrieve(data_url, 'data.json')

('data.json', <http.client.HTTPMessage at 0x7f03ff3948d0>)

In [4]:
with open('data.json', 'r') as f:
  data = json.load(f)

In [5]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for data in data['mental_healths']:
    for question in data['question']:
        word = nltk.word_tokenize(question)
        words.extend(word)        
        documents.append((word, data['tag']))
        if data['tag'] not in classes:
          classes.append(data['tag'])
print(documents)

[(['hai'], 'salam_pembuka'), (['hallo'], 'salam_pembuka'), (['apa', 'kabar', '?'], 'salam_pembuka'), (['terimakasih'], 'salam_penutup'), (['sampai', 'ketemu', 'lagi'], 'salam_penutup'), (['Apa', 'saja', 'jenis-jenis', 'gangguan', 'mental', '?'], 'pengertian_mental_health'), (['Apa', 'saja', 'gejala-gejala', 'gangguan', 'mental', '?'], 'gelaja_kesehatan_mental'), (['Apakah', 'kesehatan', 'mental', 'itu', 'penting', '?'], 'pentingnya_kesehatan_mental'), (['Bagaimana', 'cara', 'menjaga', 'kesehatan', 'mental', '?'], 'menjaga_kesehatan_mental'), (['Bagaimana', 'menjaga', 'kesehatan', 'mental', 'diusia', 'remaja', '?'], 'menjaga_kesehatan_mental_remaja'), (['Apa', 'saja', 'ciri-ciri', 'mental', 'yang', 'sehat', '?'], 'ciri_kesehatan_mental'), (['Bagaimana', 'cara', 'meningkatkan', 'self', 'awareness', '?'], 'self_awareness'), (['Bagaimana', 'melewati', 'hari', 'tanpa', 'menyakiti', 'diri', '?'], 'self_awareness'), (['Bagaimana', 'cara', 'mengendalikan', 'emosi', '?'], 'emosi'), (['Bagaimana

In [6]:
import pickle
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

36 documents
30 classes ['bipolar', 'ciri_kesehatan_mental', 'curhat', 'emosi', 'faktor_lingkungan', 'faktor_mental_health', 'faktor_resiko_gangguan_mental', 'gelaja_kesehatan_mental', 'hukum_kesehatan_jiwa', 'kesadaran_kesehatan_mental', 'keyakinan_diri', 'komunikasi', 'mengatasi_nervous', 'menjaga_kesehatan_mental', 'menjaga_kesehatan_mental_remaja', 'mental_breakdown', 'pencegahan', 'pengertian_mental_health', 'pentingnya_kesehatan_mental', 'penyebab_gangguan_mental', 'percaya_diri', 'pertolongan_pertama', 'sabar', 'salam_pembuka', 'salam_penutup', 'sedih', 'self_awareness', 'skizofrenia', 'solusi_kesehatan_mental', 'sosial_media']
113 unique lemmatized words ['ada', 'apa', 'apakah', 'atau', 'awareness', 'bagaimana', 'baik', 'bebagi', 'belum', 'bercerita', 'berkomunikasi', 'berpengaruh', 'bersabar', 'bipolar', 'breakdown', 'cara', 'cerita/curhat', 'ciri-ciri', 'curhat', 'dan', 'dapat', 'dari', 'dengan', 'dilakukan', 'dimasukan', 'diri', 'diurungkan', 'diusia', 'efficacy/keyakinan', 

In [7]:
import random
import numpy as np

training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
x_train = list(training[:,0])
y_train = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import tensorflow as tf

# model = Sequential()
# model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(len(train_y[0]), activation='softmax'))

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128,input_shape=(len(x_train[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(y_train[0]), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
8/8 [==============================] - 3s 51ms/step - loss: 3.4045 - accuracy: 0.0278
Epoch 2/200
8/8 [==============================] - 0s 47ms/step - loss: 3.3979 - accuracy: 0.0556
Epoch 3/200
8/8 [==============================] - 0s 47ms/step - loss: 3.4020 - accuracy: 0.0556
Epoch 4/200
8/8 [==============================] - 0s 47ms/step - loss: 3.4020 - accuracy: 0.0556
Epoch 5/200
8/8 [==============================] - 0s 48ms/step - loss: 3.3872 - accuracy: 0.0833
Epoch 6/200
8/8 [==============================] - 0s 44ms/step - loss: 3.3948 - accuracy: 0.0833
Epoch 7/200
8/8 [==============================] - 0s 46ms/step - loss: 3.4058 - accuracy: 0.1111
Epoch 8/200
8/8 [==============================] - 0s 44ms/step - loss: 3.3874 - accuracy: 0.0556
Epoch 9/200
8/8 [==============================] - 0s 44ms/step - loss: 3.4016 - accuracy: 0.0556
Epoch 10/200
8/8 [==============================] - 0s 47ms/step - loss: 3.3642 - accuracy: 0.0833
Epoch 11/200
8/8 [=